# LDA Training
<figure>
<img src=https://s2.loli.net/2022/02/28/X7vzOlDHJtP6UnM.png style="width: 600px">
<figcaption>The LDA training algorithm from <a href=http://www.arbylon.net/publications/text-est.pdf>Parameter estimation for text analysis</a></figcaption>
</figure>

In [39]:
%load_ext autoreload
%autoreload 2
import random
import numpy as np
from collections import defaultdict, OrderedDict
from types import SimpleNamespace
from tqdm.notebook import tqdm

In [47]:
# === corpus loading ===
class NeurIPSCorpus:
    def __init__(self, data_path, num_topics, mode, start_doc_idx=0, max_num_docs=100, max_num_words=10000, max_doc_length=1000, train_corpus=None):
        self.docs = []
        self.word2id = OrderedDict()
        self.max_doc_length = max_doc_length
        self.mode = mode

        # only keep the most frequent words
        if self.mode == "train":
            word2cnt = defaultdict(int)
            with open(data_path) as fin:
                for i, line in enumerate(fin):
                    if i >= max_num_docs: break
                    for word in line.strip().split():
                        word2cnt[word] += 1
            
            word2cnt = sorted(list(word2cnt.items()), key=lambda x: x[1], reverse=True)
            if len(word2cnt) > max_num_words:
                word2cnt = word2cnt[:max_num_words]
            word2cnt = dict(word2cnt)

        # read in the doc and convert words to integers
        with open(data_path) as fin:
            for i, line in enumerate(fin):
                if i < start_doc_idx: continue
                if i - start_doc_idx >= max_num_docs: break
                doc = []
                for word in line.strip().split():
                    if len(doc) >= self.max_doc_length: break
                    if self.mode == "train" and word not in word2cnt: continue
                    if self.mode == "train":
                        if word not in self.word2id: 
                            self.word2id[word] = len(self.word2id)
                        doc.append(self.word2id[word])
                    else:
                        if word not in train_corpus.word2id: continue
                        doc.append(train_corpus.word2id[word])
                self.docs.append(doc)

        self.num_docs = len(self.docs)
        self.num_topics = num_topics
        self.num_words = len(self.word2id)
        self.id2word = {v: k for k, v in self.word2id.items()}
        print(
            "num_docs:", self.num_docs, 
            "num_topics:", self.num_topics, 
            "num_words:", self.num_words
        )

corpus = NeurIPSCorpus(
    data_path="data/papers.txt", 
    mode="train",
    num_topics=10,
    start_doc_idx=0,
    max_num_docs=1000,
    max_num_words=10000,
    max_doc_length=200,
)
hparams = SimpleNamespace(
    alpha=np.ones([corpus.num_topics], dtype=float) / corpus.num_topics,
    beta = np.ones([corpus.num_words], dtype=float) / corpus.num_topics,
    gibbs_sampling_max_iters=1000,
)

num_docs: 1000 num_topics: 10 num_words: 7882


In [34]:
# === initialization ===
print("Initializing...", flush=True)
n_doc_topic = np.zeros([corpus.num_docs, corpus.num_topics], dtype=float) # n_m^(k)
n_topic_word = np.zeros([corpus.num_topics, corpus.num_words], dtype=float) # n_k^(t)
z_doc_word = np.zeros([corpus.num_docs, corpus.max_doc_length], dtype=int)

for doc_i in range(corpus.num_docs):
    for j, word_j in enumerate(corpus.docs[doc_i]):
        topic_ij = random.randint(0, corpus.num_topics - 1)
        n_doc_topic[doc_i, topic_ij] += 1
        n_topic_word[topic_ij, word_j] += 1
        z_doc_word[doc_i, j] = topic_ij

# === Gibbs sampling ===
print("Gibbs sampling...", flush=True)
for iteration in tqdm(range(hparams.gibbs_sampling_max_iters)):
    for doc_i in range(corpus.num_docs):
        for j, word_j in enumerate(corpus.docs[doc_i]):
            # remove the old assignment
            topic_ij = z_doc_word[doc_i, j]
            n_doc_topic[doc_i, topic_ij] -= 1
            n_topic_word[topic_ij, word_j] -= 1
            # compute the new assignment
            p_doc_topic = (n_doc_topic[doc_i, :] + hparams.alpha) \
                        / np.sum(n_doc_topic[doc_i] + hparams.alpha)
            p_topic_word = (n_topic_word[:, word_j] + hparams.beta[word_j]) \
                        / np.sum(n_topic_word + hparams.beta, axis=1)
            p_topic = p_doc_topic * p_topic_word
            p_topic /= np.sum(p_topic)
            # record the new assignment
            new_topic_ij = np.random.choice(np.arange(corpus.num_topics), p=p_topic)
            n_doc_topic[doc_i, new_topic_ij] += 1
            n_topic_word[new_topic_ij, word_j] += 1
            z_doc_word[doc_i, j] = new_topic_ij

    if iteration % 10 == 0:
        print(f"Iter [{iteration}]===")
        # === Check convergence and read out parameters ===
        theta = (n_doc_topic + hparams.alpha) / np.sum(n_doc_topic + hparams.alpha, axis=1, keepdims=True)
        phi = (n_topic_word + hparams.beta) / np.sum(n_topic_word + hparams.beta, axis=1, keepdims=True)

        for topic in range(corpus.num_topics):
            top_words = np.argsort(phi[topic])[::-1][:10]
            top_probs = phi[topic, top_words]
            top_words = [corpus.id2word[word] for word in top_words]
            print(f"Topic {topic}:", top_words)

Initializing...
Gibbs sampling...


Iter [0]===
Topic 0: ['model', 'learning', 'method', 'problem', 'network', 'algorithm', 'abstract', 'datum', 'neural', 'system']
Topic 1: ['model', 'learning', 'algorithm', 'network', 'neural', 'system', 'function', 'information', 'approach', 'base']
Topic 2: ['network', 'model', 'problem', 'algorithm', 'datum', 'learning', 'neural', 'abstract', 'function', 'university']
Topic 3: ['model', 'network', 'neural', 'system', 'algorithm', 'function', 'problem', 'learn', 'learning', 'datum']
Topic 4: ['network', 'model', 'algorithm', 'neural', 'problem', 'function', 'datum', 'learning', 'information', 'e']
Topic 5: ['model', 'network', 'algorithm', 'neural', 'learning', 'input', 'abstract', 'introduction', 'function', 'set']
Topic 6: ['model', 'network', 'problem', 'algorithm', 'system', 'neural', 'base', 'learn', 'approach', 'set']
Topic 7: ['model', 'network', 'method', 'learning', 'neural', 'system', 'function', 'datum', 'algorithm', 'problem']
Topic 8: ['model', 'problem', 'network', 'alg

In [67]:
from visualize import visualize_topic_word

all_top_words = []
all_top_probs = []
for topic in range(corpus.num_topics):
    top_words = np.argsort(phi[topic])[::-1][:10]
    top_probs = phi[topic, top_words]
    top_words = [corpus.id2word[word] for word in top_words]
    all_top_words.append(top_words)
    all_top_probs.append(top_probs)
    print(f"Topic {topic}:", top_words)
visualize_topic_word(all_top_words, all_top_probs)

Topic 0: ['learning', 'problem', 'state', 'control', 'learn', 'reinforcement', 'algorithm', 'decision', 'time', 'value']
Topic 1: ['neuron', 'model', 'cell', 'input', 'stimulus', 'neural', 'spike', 'information', 'response', 'activity']
Topic 2: ['image', 'object', 'visual', 'model', 'human', 'motion', 'task', 'face', 'recognition', 'system']
Topic 3: ['datum', 'algorithm', 'problem', 'vector', 'method', 'feature', 'set', 'space', 'classification', 'class']
Topic 4: ['network', 'learning', 'function', 'algorithm', 'neural', 'learn', 'weight', 'training', 'error', 'input']
Topic 5: ['network', 'neural', 'system', 'time', 'analog', 'circuit', 'state', 'architecture', 'chip', 'computer']
Topic 6: ['signal', 'source', 'code', 'component', 'noise', 'auditory', 'independent', 'information', 'system', 'analysis']
Topic 7: ['model', 'datum', 'distribution', 'probability', 'gaussian', 'variable', 'method', 'bayesian', 'mixture', 'algorithm']
Topic 8: ['x', 't', 'e', 'j', 'n', 'f', 'y', 'l', 'p'

In [53]:
# === inference on unseen documents ===
test_corpus = NeurIPSCorpus(
    data_path="data/papers.txt", 
    mode="test",
    num_topics=10,
    start_doc_idx=1000,
    max_num_docs=5,
    max_num_words=10000,
    max_doc_length=200,
    train_corpus=corpus,
)
for i, doc in enumerate(test_corpus.docs):
    print(f"Test Doc [{i}] ===")
    doc_i = 0   # only infer 1 test doc at a time
    test_n_doc_topic = np.zeros([1, corpus.num_topics], dtype=float)
    test_n_topic_word = np.zeros([corpus.num_topics, corpus.num_words], dtype=float)
    test_z_doc_word = np.zeros([1, corpus.max_doc_length], dtype=int)

    print(" ".join([corpus.id2word[x] for x in doc]))

    for j, word_j in enumerate(doc):
        topic_ij = random.randint(0, corpus.num_topics - 1)
        test_n_doc_topic[doc_i, topic_ij] += 1
        test_n_topic_word[topic_ij, word_j] += 1
        test_z_doc_word[doc_i, j] = topic_ij

    for iteration in tqdm(range(100)):
        for j, word_j in enumerate(doc):
            # remove the old assignment
            topic_ij = test_z_doc_word[doc_i, j]
            test_n_doc_topic[doc_i, topic_ij] -= 1
            test_n_topic_word[topic_ij, word_j] -= 1
            # compute the new assignment
            p_doc_topic = (test_n_doc_topic[doc_i, :] + hparams.alpha) \
                        / np.sum(test_n_doc_topic[doc_i] + hparams.alpha)
            p_topic_word = (test_n_topic_word[:, word_j] + n_topic_word[:, word_j] + hparams.beta[word_j]) \
                        / np.sum(test_n_topic_word + n_topic_word + hparams.beta, axis=1)
            p_topic = p_doc_topic * p_topic_word
            p_topic /= np.sum(p_topic)
            # record the new assignment
            new_topic_ij = np.random.choice(np.arange(corpus.num_topics), p=p_topic)
            test_n_doc_topic[doc_i, new_topic_ij] += 1
            test_n_topic_word[new_topic_ij, word_j] += 1
            test_z_doc_word[doc_i, j] = new_topic_ij

    # === Check convergence and read out parameters ===
    test_theta = (test_n_doc_topic + hparams.alpha) / np.sum(test_n_doc_topic + hparams.alpha, axis=1, keepdims=True)
    test_phi = (test_n_topic_word + hparams.beta) / np.sum(test_n_topic_word + hparams.beta, axis=1, keepdims=True)
    print("Topic distribution:", test_theta)
    print("Top 3 topics:", np.argsort(test_theta[0])[::-1][:3])

num_docs: 5 num_topics: 10 num_words: 0
Test Doc [0] ===
bind graphical model m a r h j not department biophysics university nijmegen nl ez nijmegen netherlands bert kun nl abstract present method bound partition function boltzmann machine neural network odd order polynomial direct extension mean field bind order order bind strictly well mean field additionally rough outline bound applicable sigmoid belief network numerical experiment indicate error reduction factor easily reach region expansion base approximation useful introduction graphical model capability model large class probability distribution neuron network random variable connection model causal dependency usually node direct relation random variable problem call visible node know hidden model complex probability distribution learn graphical model long likelihood visible correspond pattern data set compute general time take scale exponentially number hide neuron architecture choice approximation likelihood know approximation


Topic distribution: [[4.97512438e-04 1.54228856e-02 4.97512438e-04 5.02487562e-02
  3.28855721e-01 2.03980100e-02 4.97512438e-04 5.82587065e-01
  4.97512438e-04 4.97512438e-04]]
Top 3 topics: [7 4 3]
Test Doc [1] ===
decomposition reinforcement learning admission control self similar arrival processes department electrical engineering technion haifa israel ee technion ac il abstract paper present predictive gain scheduling technique simplify reinforcement learning problem decomposition link admission control self similar traffic demonstrate technique control problem decompose line prediction near future arrival rate policy poisson arrival process decision time prediction select policy simulation technique result significantly fast learn performance loss compare reinforcement learn controller decompose problem introduction multi service communication network asynchronous transfer mode network resource control crucial importance network operator user objective maintain service quality m


Topic distribution: [[5.92537313e-01 1.04975124e-01 1.04477612e-02 4.97512438e-04
  6.51741294e-02 5.47263682e-03 4.97512438e-04 4.97512438e-04
  4.97512438e-04 2.19402985e-01]]
Top 3 topics: [0 9 1]
Test Doc [2] ===
model spatial recall mental imagery neglect becker department psychology university main street west hamilton canada l s kl becker ca burgess department anatomy institute cognitive neuroscience ucl queen square london uk wcin ar n burgess ucl ac uk abstract present computational model neural mechanism parietal temporal lobe support spatial navigation recall scene imagery product recall long term representation store hippocampus associate local spatial object relate feature region viewer center representation dynamically generate long term memory parietal model model simulate recall imagery location object complex environment parietal damage model exhibit neglect mental imagery rotate imagine perspective observer square experiment model make novel prediction neural represe


Topic distribution: [[0.01542289 0.44328358 0.4681592  0.01542289 0.00049751 0.00049751
  0.00049751 0.00049751 0.00049751 0.05522388]]
Top 3 topics: [2 1 9]
Test Doc [3] ===
new approaches robust adaptive speech recognition bourlard bengio weber p o box du switzerland bourlard bengio weber ch abstract paper discuss new research direction automatic speech recognition asr somewhat deviate usual approach specifically motivate briefly describe new approach base multi stream multi band asr approach extend standard hide markov model hmm base approach assume different frequency channel represent speech signal process different independent expert expert focus different characteristic signal different stream likelihood posterior combine temporal stage yield global recognition output extension multi stream asr finally introduce new approach refer hmm hmm emission probability estimate state specific feature base hmm responsible merge stream information model possible correlation multi channel p


Topic distribution: [[0.00049751 0.00049751 0.060199   0.00547264 0.00049751 0.12985075
  0.15970149 0.20447761 0.00049751 0.43830846]]
Top 3 topics: [9 7 6]
Test Doc [4] ===
variational mean field theory sigmoidal belief network c bhattacharyya computer science automation indian institute science in s mechanical production engineering national university singapore edu abstract variational derivation plefka mean field theory present theory apply sigmoidal belief network aid approximation empirical evaluation small scale network propose approximation competitive introduction application mean field theory solve problem inference belief networks know paper discuss variational mean field theory application bn sigmoidal bn particular present variational derivation mean field theory propose plefka theory develop stochastic system n binary random variable si e o describe energy function e s follow boltzmann gibbs distribution temperature t p s e z z t e s e y s application mean field method 


Topic distribution: [[4.97512438e-04 4.97512438e-04 4.97512438e-04 4.97512438e-04
  1.24875622e-01 5.02487562e-02 4.97512438e-04 5.42786070e-01
  2.69154229e-01 1.04477612e-02]]
Top 3 topics: [7 8 4]
